In [1]:
# Importing related libraries

from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import re
import time

### Determination of House Links' Data

In [2]:
with open('item.html', 'r', encoding='UTF8') as f:
    item = f.read()

text_html = bs(item, parser='html.parser')

In [3]:
'https://www.century21global.com'+text_html.body.a['href']

'https://www.century21global.com/en/p/13123-sw-41st-pl-highway-ocala-florida-34481-usa--rl-133244110'

In [4]:
text_html.body.a['href'].split('-')[-1]

'133244110'

In [10]:
text_html.a.div.div.div.next_sibling.div.next_sibling.next_sibling.text

'$33,000,000 USD'

### Scraping House Links' Data

In [15]:
homes_for_sale_links = []
total_pages = 201

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
driver_path = Service('<your_chromedriver_file_path>')

driver = webdriver.Chrome(service=driver_path, options=chrome_options)

for h in range(1, total_pages):

    # The link text may be updated according to your filter
    driver.get('https://www.century21global.com/en/l/homes-for-sale/usa?max=50&priceFrom=200000&'\
               'sort=PRICE&order=DESC&priceTo=1000000&bedrooms=1&bathrooms=1&propertySizeFrom=50&'\
                'propertySizeTo=300&page='+str(h))
    time.sleep(np.random.randint(3, 6))

    homes_raw = driver.find_elements(By.CSS_SELECTOR, 'div[class="col-lg-6 col-xl-4 ng-star-inserted"]')

    for home in homes_raw:

        link_html = bs(home.get_attribute('innerHTML'), 'html.parser')

        try:
            home_id = link_html.a['href'].split('-')[-1]
        except:
            home_id = None

        try:
            price = link_html.a.div.div.div.next_sibling.div.next_sibling.next_sibling.text
        except:
            price = None

        try:
            link = 'https://www.century21global.com'+link_html.a['href']
        except:
            link = None
    
        homes_for_sale_links.append([home_id, price, link])
        
    
    print(f'page {h} completed')
    clear_output(wait=True)

    # Saving data every 25 pages in case of any interruption. May be skipped
    if h % 25 == 0:
        df_links = pd.DataFrame(data=homes_for_sale_links, columns=['home_id', 'price', 'link'])
        df_links.to_csv('your_links_file_name.csv', index=False)
    else:
        continue

driver.close()

# Saving whole data
df_links = pd.DataFrame(data=homes_for_sale_links, columns=['home_id', 'price', 'link'])
df_links.to_csv('your_links_file_name.csv', index=False)

print(f'All process completed.')

All process completed.


### Determination of Houses' Data

In [16]:
with open('item2.html', 'r', encoding='UTF8') as f:
    item = f.read()

text_html2 = bs(item, parser='html.parser')

In [17]:
text_html2.div.h2.next_sibling.text.split(': ')[1]

'Apr 3, 2024'

In [110]:
text_html2.find('div', class_='mb-5 ng-star-inserted').text.split(': ')[1]

'Apr 3, 2024'

In [111]:
address_elements = text_html2.find('h1', class_='mat-h4 mb-0 ng-star-inserted').contents

In [112]:
', '.join([a.text for a in address_elements[1].contents if a != ' '])

'Cape Coral, Florida, United States, 33904'

In [113]:
address_elements[0].text

'5769 Riverside Drive'

In [114]:
address_elements[0].text+', '+', '.join([a.text for a in address_elements[1].contents if a != ' '])

'5769 Riverside Drive, Cape Coral, Florida, United States, 33904'

In [115]:
re.sub('\n+', ', ', text_html2.find('div', class_='row data-list').text.strip().title())

'River, Fire Pit, Water View'

In [116]:
text_html2.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')[2].text.title()

' Area  Property Size  4,967Ft²  Land/Lot Size  11,761Ft²  (0.27Ac)  Bedrooms 5 Bathrooms  7  Cooling System  Central Air Conditioning-Electric  Central Ac '

In [117]:
re.findall('Year Built (\d+) ', text_html2.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')[0].text.title())[0]

'2016'

In [118]:
re.findall('Size  (.*)Ft²', text_html2.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')[2].text.title())[0]

'4,967Ft²  Land/Lot Size  11,761'

In [119]:
text_html2.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')[2].contents[1].find_all('dt')

[<dt class="ng-star-inserted"> Property size </dt>,
 <dt class="ng-star-inserted"> Land/Lot size </dt>,
 <dt class="ng-star-inserted"> Bedrooms </dt>,
 <dt class="ng-star-inserted"> Bathrooms </dt>,
 <dt class="ng-star-inserted"> Cooling system </dt>]

In [120]:
vert_dls = text_html2.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')

keys = [item for items in vert_dls for item in items.contents[1].find_all('dt')]
values = [item for items in vert_dls for item in items.contents[1].find_all('dd')]

In [121]:
keys

[<dt class="ng-star-inserted"> Year built </dt>,
 <dt class="ng-star-inserted"> Roofing </dt>,
 <dt class="ng-star-inserted"> Property features </dt>,
 <dt class="ng-star-inserted"> Appliances </dt>,
 <dt class="ng-star-inserted"> Property size </dt>,
 <dt class="ng-star-inserted"> Land/Lot size </dt>,
 <dt class="ng-star-inserted"> Bedrooms </dt>,
 <dt class="ng-star-inserted"> Bathrooms </dt>,
 <dt class="ng-star-inserted"> Cooling system </dt>]

In [122]:
values

[<dd class="ng-star-inserted">2016</dd>,
 <dd class="comma-separated-fragments ng-star-inserted"><div class="fragment ng-star-inserted"> metal </div><!-- --></dd>,
 <dd class="ng-star-inserted"><div class="comma-separated-fragments"><div class="fragment ng-star-inserted"> Porch </div><div class="fragment ng-star-inserted"> garage </div><div class="fragment ng-star-inserted"> Balcony </div><div class="fragment ng-star-inserted"> Deck </div><!-- --></div></dd>,
 <dd class="comma-separated-fragments ng-star-inserted"><div class="fragment ng-star-inserted"> Washer </div><div class="fragment ng-star-inserted"> Double Oven </div><div class="fragment ng-star-inserted"> Microwave </div><div class="fragment ng-star-inserted"> Refrigerator </div><div class="fragment ng-star-inserted"> Garbage Disposal </div><div class="fragment ng-star-inserted"> Dryer </div><div class="fragment ng-star-inserted"> Dishwasher </div><!-- --></dd>,
 <dd class="ng-star-inserted"> 4,967ft² <!-- --><!-- --><!-- --><!-

In [123]:
{k.text.strip(): v.text.strip() for k, v in zip(vert_dls[0].contents[1].find_all('dt'),
                                                vert_dls[0].contents[1].find_all('dd'))}

{'Year built': '2016', 'Roofing': 'metal'}

In [124]:
{k.text.strip(): v.text.strip() for k, v in zip(vert_dls[1].contents[1].find_all('dt'),
                                                vert_dls[1].contents[1].find_all('dd'))}

{'Property features': 'Porch  garage  Balcony  Deck',
 'Appliances': 'Washer  Double Oven  Microwave  Refrigerator  Garbage Disposal  Dryer  Dishwasher'}

In [126]:
{k.text.strip(): v.text.strip() for k, v in zip(vert_dls[2].contents[1].find_all('dt'),
                                                vert_dls[2].contents[1].find_all('dd'))}

{'Property size': '4,967ft²',
 'Land/Lot size': '11,761ft²  (0.27ac)',
 'Bedrooms': '5',
 'Bathrooms': '7',
 'Cooling system': 'Central Air Conditioning-Electric  Central AC'}

In [127]:
{k.text.strip(): v.text.strip() for k, v in zip(keys, values)}

{'Year built': '2016',
 'Roofing': 'metal',
 'Property features': 'Porch  garage  Balcony  Deck',
 'Appliances': 'Washer  Double Oven  Microwave  Refrigerator  Garbage Disposal  Dryer  Dishwasher',
 'Property size': '4,967ft²',
 'Land/Lot size': '11,761ft²  (0.27ac)',
 'Bedrooms': '5',
 'Bathrooms': '7',
 'Cooling system': 'Central Air Conditioning-Electric  Central AC'}

In [128]:
text_html2.find('p', class_='opacity-75 ng-star-inserted').text

"Luxury meets waterfront living in this 2-story contemporary masterpiece located in Cape Coral's coveted Yacht Club area. With almost 5000 sq ft of interior living space, this 5 bedrooms, 7 bathrooms, and a 3-car garage home offers unparalleled elegance and panoramic views of the Caloosahatchee River. Key features include a gourmet kitchen with an induction cooktop, a cappuccino maker, and a temperature-controlled wine room. The resort-style pool overlooks the river, providing the perfect setting for outdoor living. Boasting unique wall coverings, fixtures, and furnishings throughout, this fully furnished home is move-in ready. Enjoy sunrises and sunsets from the glass-walled great room or the outdoor seating area with a fire pit. Boating enthusiasts will appreciate the 20,000 lb covered boat lift, providing quick access to world-renowned beaches. With two master suites on the second level, a loft, and a workout area with a private balcony, this home is designed for luxurious living. C

### Scraping House' Data

In [129]:
df_links = pd.read_csv('homes_for_sale_links.csv')

df_links['link']

0       https://www.century21global.com/en/p/3910-n-st...
1       https://www.century21global.com/en/p/1569-238t...
2       https://www.century21global.com/en/p/1569-238t...
3       https://www.century21global.com/en/p/1848-tier...
4       https://www.century21global.com/en/p/19633-est...
                              ...                        
9995    https://www.century21global.com/en/p/613-jacks...
9996    https://www.century21global.com/en/p/16775-shr...
9997    https://www.century21global.com/en/p/319-e-13t...
9998    https://www.century21global.com/en/p/489-wolco...
9999    https://www.century21global.com/en/p/751-labar...
Name: link, Length: 10000, dtype: object

In [130]:
df_links['home_id'].nunique()

9870

In [141]:
# Dropping dublicated links

df_links.drop_duplicates(inplace=True, ignore_index=True)

df_links

,home_id,price,link
0,132976751,"$1,000,000 USD",https://www.century21global.com/en/p/3910-n-st...
1,133515366,"$1,000,000 USD",https://www.century21global.com/en/p/1569-238t...
2,133515346,"$1,000,000 USD",https://www.century21global.com/en/p/1569-238t...
3,133753203,"$1,000,000 USD",https://www.century21global.com/en/p/1848-tier...
4,133724019,"$1,000,000 USD",https://www.century21global.com/en/p/19633-est...
...,...,...,...
9865,133703887,"$219,000 USD",https://www.century21global.com/en/p/613-jacks...
9866,128732081,"$219,000 USD",https://www.century21global.com/en/p/16775-shr...
9867,133826856,"$219,000 USD",https://www.century21global.com/en/p/319-e-13t...
9868,133783603,"$219,000 USD",https://www.century21global.com/en/p/489-wolco...


In [142]:
homes_for_sale = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
driver_path = Service('<your_chromedriver_file_path>')

driver = webdriver.Chrome(service=driver_path, options=chrome_options)

for idx, h in enumerate(df_links['link']):

    driver.get(h)
    time.sleep(np.random.randint(3, 5))
    home_raw = driver.find_element(By.CSS_SELECTOR, 'div.main-content')

    home_html = bs(home_raw.get_attribute('innerHTML'), 'html.parser')

    home_id = df_links['home_id'][idx]

    try:
        date_updated = home_html.find('div', class_='mb-5 ng-star-inserted').text.split(': ')[1]
    except:
        date_updated = None

    price = df_links['price'][idx]

    try:
        address_elements = home_html.find('h1', class_='mat-h4 mb-0 ng-star-inserted').contents
        address = address_elements[0].text+', '+', '.join([a.text for a in address_elements[1].contents if a != ' '])
    except:
        address = None

    try:
        key_features = re.sub('\n+', ', ', home_html.find('div', class_='row data-list').text.strip().title())
    except:
        key_features = None
    
    try:
        vert_dls = home_html.find_all('div', class_='mb-5 col-sm-6 col-md-4 ng-star-inserted')
        keys = [item for items in vert_dls for item in items.contents[1].find_all('dt')]
        values = [item for items in vert_dls for item in items.contents[1].find_all('dd')]

        features = {k.text.strip(): v.text.strip() for k, v in zip(keys, values)}
    except:
        features = None
    
    try:
        desc = home_html.find('p', class_='opacity-75 ng-star-inserted').text
    except:
        desc = None

    homes_for_sale.append([home_id, date_updated, price, address, key_features, features, desc])
    
    print(f'item {idx+1} completed')
    clear_output(wait=True)

    # Saving data every 500 houses in case of any interruption. May be skipped
    if (idx+1) % 500 == 0:
        df_homes = pd.DataFrame(data=homes_for_sale,
                                columns=['home_id', 'date_updated',
                                         'price', 'address',
                                         'key_features', 'features', 'desc'])
        df_homes.to_csv('your_homes_file_name.csv', index=False)
    else:
        continue

driver.close()

# Saving whole data
df_homes = pd.DataFrame(data=homes_for_sale,
                        columns=['home_id', 'date_updated',
                                    'price', 'address',
                                    'key_features', 'features', 'desc'])
df_homes.to_csv('your_homes_file_name.csv', index=False)

print(f'All process completed.')

All process completed.


### Data cleansing is in cleaning.ipynb file